In [1]:
from utils import start_environment

In [2]:
start_environment()

In [3]:

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
from keras.layers import LSTM, GRU


input_length = [100]
load_emb = [False]
emb_size = [300]
vocabulary_length = [5000]
trainable_emb = [True]
cnn_size = [[None]] # [[128], [128, 128], [512]]
cnn_filter = [[3]]
rnn_size = [[64], [128]]
cell_type = [LSTM, GRU]
bidirectional = [False]
attention = [True, False]
dropout = [0.5]
dnn_size = [[32], [128, 32], [512]]
batch_size = [1024]

param   = [input_length, load_emb, emb_size, vocabulary_length, trainable_emb, cnn_size, cnn_filter, rnn_size, cell_type, bidirectional, attention, dropout, dnn_size, batch_size]
indexes = ["input_length", "load_emb", "emb_size", "vocabulary_length", "trainable_emb", "cnn_size", "cnn_filter", "rnn_size", "cell_type", "bidirectional", "attention", "dropout", "dnn_size", "batch_size"]


In [6]:
from dl_models import ModelExperiments

In [7]:
me = ModelExperiments(param, indexes)

In [8]:
me.compile_models()

KeyError: 'embedding_matrix'